In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import data
obs_A = pd.read_parquet('../../preprocessing/data/obs_A.parquet')
obs_B = pd.read_parquet('../../preprocessing/data/obs_B.parquet')
obs_C = pd.read_parquet('../../preprocessing/data/obs_C.parquet')
est_A = pd.read_parquet('../../preprocessing/data/est_A.parquet')
est_B = pd.read_parquet('../../preprocessing/data/est_B.parquet')
est_C = pd.read_parquet('../../preprocessing/data/est_C.parquet')
test_A = pd.read_parquet('../../preprocessing/data/test_A.parquet')
test_B = pd.read_parquet('../../preprocessing/data/test_B.parquet')
test_C = pd.read_parquet('../../preprocessing/data/test_C.parquet')

obs_A.columns = [col.replace('[', '').replace(']', '').replace(',', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace(':', '').replace('\\', '') for col in obs_A.columns]
est_A.columns = [col.replace('[', '').replace(']', '').replace(',', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace(':', '').replace('\\', '') for col in est_A.columns]
test_A.columns = [col.replace('[', '').replace(']', '').replace(',', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace(':', '').replace('\\', '') for col in test_A.columns]

obs_B.columns = [col.replace('[', '').replace(']', '').replace(',', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace(':', '').replace('\\', '') for col in obs_B.columns]
est_B.columns = [col.replace('[', '').replace(']', '').replace(',', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace(':', '').replace('\\', '') for col in est_B.columns]
test_B.columns = [col.replace('[', '').replace(']', '').replace(',', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace(':', '').replace('\\', '') for col in test_B.columns]

obs_C.columns = [col.replace('[', '').replace(']', '').replace(',', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace(':', '').replace('\\', '') for col in obs_C.columns]
est_C.columns = [col.replace('[', '').replace(']', '').replace(',', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace(':', '').replace('\\', '') for col in est_C.columns]
test_C.columns = [col.replace('[', '').replace(']', '').replace(',', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace(':', '').replace('\\', '') for col in test_C.columns]

In [3]:
# Concatinate
A = pd.concat([obs_A, est_A])
B = pd.concat([obs_B, est_B])
C = pd.concat([obs_C, est_C])

# Data splits for submissions
X_A = A.drop(columns='pv_measurement')
y_A = A['pv_measurement']
X_B = B.drop(columns='pv_measurement')
y_B = B['pv_measurement']
X_C = C.drop(columns='pv_measurement')
y_C = C['pv_measurement']

# Data splits for testing
# train_A, test_A = train_test_split(A, test_size=0.2, shuffle=True, random_state=42)
# X_train_A = train_A.drop(columns='pv_measurement')
# y_train_A = train_A['pv_measurement']
# X_test_A = test_A.drop(columns='pv_measurement')
# y_test_A = test_A['pv_measurement']

# train_B, test_B = train_test_split(B, test_size=0.2, shuffle=True, random_state=42)
# X_train_B = train_B.drop(columns='pv_measurement')
# y_train_B = train_B['pv_measurement']
# X_test_B = test_B.drop(columns='pv_measurement')
# y_test_B = test_B['pv_measurement']

# train_C, test_C = train_test_split(C, test_size=0.2, shuffle=True, random_state=42)
# X_train_C = train_C.drop(columns='pv_measurement')
# y_train_C = train_C['pv_measurement']
# X_test_C = test_C.drop(columns='pv_measurement')
# y_test_C = test_C['pv_measurement']



In [4]:
# Inspect data
# X_train_A.info()

In [5]:
# XGBoost parameters
xgb_parameters          = {
                            'colsample_bytree': 0.8, 
                            'gamma': 0.8, 
                            'learning_rate': 0.008, 
                            'max_depth': 15, 
                            'min_child_weight': 10, 
                            'n_estimators': 800, 
                            'reg_alpha': 1, 
                            'reg_lambda': 3, 
                            'subsample': 0.912,
                            'random_state': 0, 
                            'booster': 'gbtree'
                        }


catboost_parameters     = { 
                            'subsample': 0.7222222222222222, 
                            'rsm': 0.6111111111111112, 
                            'random_strength': 0.4111111111111111, 
                            'min_data_in_leaf': 19, 
                            'learning_rate': 0.020000000000000004, 
                            'l2_leaf_reg': 1.0, 
                            'iterations': 750, 
                            'grow_policy': 'Depthwise', 
                            'depth': 10, 
                            'border_count': 115, 
                            'bootstrap_type': 'MVS' }

lgbm_parameters         = { 
                            'boosting_type': 'dart',
                            'objective': 'regression',
                            'metric': 'mae',
                            'num_leaves': 100,
                            'learning_rate': 0.05,
                            'verbose': 1,
                            'num_iterations': 300 }
# Define base models
base_models = [
    ('xgb', XGBRegressor(**xgb_parameters, n_jobs=-1)),
    ('cat', CatBoostRegressor(**catboost_parameters)),
    ('lgbm', LGBMRegressor(**lgbm_parameters, n_jobs=-1))
]

# Define meta-model
meta_model = Ridge()

# Define stacking ensemble
stack_A = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=5, verbose=1)
stack_B = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=5, verbose=1)
stack_C = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=5, verbose=1)

In [6]:
# Fit stacking ensemble
stack_A.fit(X_A, y_A)
pickle.dump(stack_A, open('./stack_models/stack_A.pkl', 'wb'))

stack_B.fit(X_B, y_B)
pickle.dump(stack_B, open('./stack_models/stack_B.pkl', 'wb'))

stack_C.fit(X_C, y_C)
pickle.dump(stack_C, open('./stack_models/stack_C.pkl', 'wb'))

0:	learn: 1146.5624810	total: 84.7ms	remaining: 1m 3s
1:	learn: 1127.5759891	total: 113ms	remaining: 42.4s
2:	learn: 1109.0399276	total: 142ms	remaining: 35.3s
3:	learn: 1090.9191030	total: 167ms	remaining: 31.1s
4:	learn: 1073.1135288	total: 195ms	remaining: 29.1s
5:	learn: 1055.7441300	total: 221ms	remaining: 27.4s
6:	learn: 1038.7279548	total: 252ms	remaining: 26.8s
7:	learn: 1022.0991908	total: 287ms	remaining: 26.6s
8:	learn: 1005.8666216	total: 316ms	remaining: 26s
9:	learn: 989.9881667	total: 344ms	remaining: 25.4s
10:	learn: 974.5443370	total: 381ms	remaining: 25.6s
11:	learn: 959.3083428	total: 417ms	remaining: 25.7s
12:	learn: 944.4801855	total: 438ms	remaining: 24.8s
13:	learn: 929.9673690	total: 470ms	remaining: 24.7s
14:	learn: 915.7652883	total: 501ms	remaining: 24.6s
15:	learn: 901.9593604	total: 543ms	remaining: 24.9s
16:	learn: 888.3972340	total: 606ms	remaining: 26.1s
17:	learn: 875.2387956	total: 637ms	remaining: 25.9s
18:	learn: 862.2657407	total: 672ms	remaining: 2

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 12.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 1174.5945761	total: 21.3ms	remaining: 16s
1:	learn: 1154.9765122	total: 44.6ms	remaining: 16.7s
2:	learn: 1135.9423719	total: 66.6ms	remaining: 16.6s
3:	learn: 1117.1986082	total: 91.1ms	remaining: 17s
4:	learn: 1098.8563724	total: 117ms	remaining: 17.5s
5:	learn: 1081.0672673	total: 139ms	remaining: 17.2s
6:	learn: 1063.5863332	total: 161ms	remaining: 17.1s
7:	learn: 1046.3559491	total: 186ms	remaining: 17.3s
8:	learn: 1029.5463306	total: 216ms	remaining: 17.8s
9:	learn: 1013.2010763	total: 242ms	remaining: 17.9s
10:	learn: 997.2691562	total: 268ms	remaining: 18s
11:	learn: 981.6612530	total: 289ms	remaining: 17.8s
12:	learn: 966.3952323	total: 310ms	remaining: 17.6s
13:	learn: 951.4447543	total: 340ms	remaining: 17.9s
14:	learn: 936.7241050	total: 366ms	remaining: 17.9s
15:	learn: 922.3085486	total: 393ms	remaining: 18s
16:	learn: 908.3045819	total: 417ms	remaining: 18s
17:	learn: 894.5695153	total: 441ms	remaining: 17.9s
18:	learn: 881.0938913	total: 473ms	remaining: 18.2s

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014053 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3420
[LightGBM] [Info] Number of data points in the train set: 27268, number of used features: 19
[LightGBM] [Info] Start training from score 592.508243
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020135 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3414
[LightGBM] [Info] Number of data points in the train set: 27268, number of used features: 19
[LightGBM] [Info] Start training from score 586.618095
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004384 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 175.3721976	total: 18.9ms	remaining: 14.2s
1:	learn: 172.3472781	total: 36.3ms	remaining: 13.6s
2:	learn: 169.3910554	total: 55.4ms	remaining: 13.8s
3:	learn: 166.4803985	total: 70.9ms	remaining: 13.2s
4:	learn: 163.6513937	total: 89.5ms	remaining: 13.3s
5:	learn: 160.8667055	total: 107ms	remaining: 13.3s
6:	learn: 158.1510723	total: 126ms	remaining: 13.4s
7:	learn: 155.4801708	total: 146ms	remaining: 13.5s
8:	learn: 152.8493228	total: 164ms	remaining: 13.5s
9:	learn: 150.2941679	total: 181ms	remaining: 13.4s
10:	learn: 147.7866068	total: 198ms	remaining: 13.3s
11:	learn: 145.3334513	total: 217ms	remaining: 13.3s
12:	learn: 142.9357412	total: 231ms	remaining: 13.1s
13:	learn: 140.5771417	total: 250ms	remaining: 13.1s
14:	learn: 138.2620659	total: 264ms	remaining: 12.9s
15:	learn: 136.0202909	total: 283ms	remaining: 13s
16:	learn: 133.8091118	total: 307ms	remaining: 13.2s
17:	learn: 131.6494621	total: 324ms	remaining: 13.2s
18:	learn: 129.5295911	total: 340ms	remaining: 13.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   49.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002420 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3413
[LightGBM] [Info] Number of data points in the train set: 20876, number of used features: 19
[LightGBM] [Info] Start training from score 58.768313
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003150 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3415
[LightGBM] [Info] Number of data points in the train set: 20876, number of used features: 19
[LightGBM] [Info] Start training from score 78.078200
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002183 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory i

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.7min finished


In [7]:
# Evaluate
# print('MAE A:', mean_absolute_error(y_test_A, stack_A.predict(X_test_A)))
# print('MAE B:', mean_absolute_error(y_test_B, stack_B.predict(X_test_B)))
# print('MAE C:', mean_absolute_error(y_test_C, stack_C.predict(X_test_C)))

# # Total MAE for all three locations
# print('Total MAE:', (mean_absolute_error(y_test_A, stack_A.predict(X_test_A)) + mean_absolute_error(y_test_B, stack_B.predict(X_test_B)) + mean_absolute_error(y_test_C, stack_C.predict(X_test_C)))/3)

In [8]:
# Create submission

output_file = 'stack_submission.csv'

pred_A = stack_A.predict(test_A)
pred_B = stack_B.predict(test_B)
pred_C = stack_C.predict(test_C)

pred_A = np.clip(pred_A, 0, None)
pred_B = np.clip(pred_B, 0, None)
pred_C = np.clip(pred_C, 0, None)

# Concatenate predictions
predictions = np.concatenate([pred_A, pred_B, pred_C])

# Create an id array
ids = np.arange(0, len(predictions))

# Create a DataFrame
df = pd.DataFrame({
    'id': ids,
    'prediction': predictions
})

# Save to CSV
df.to_csv(output_file, index=False)
print(f"Submission saved to {output_file}")

Submission saved to stack_submission.csv
